In [1]:
import pandas as pd
from sodapy import Socrata

import matplotlib.pyplot as plt
from datetime import datetime

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("health.data.ny.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(health.data.ny.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("xdss-u53e") #limit = 62 counties in NY

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [2]:
results_df.head()

,test_date,county,new_positives,cumulative_number_of_positives,total_number_of_tests,cumulative_number_of_tests
0,2020-04-05T00:00:00.000,Albany,14,319,89,4244
1,2020-04-05T00:00:00.000,Allegany,1,17,8,196
2,2020-04-05T00:00:00.000,Bronx,959,14327,1628,27371
3,2020-04-05T00:00:00.000,Broome,5,76,33,607
4,2020-04-05T00:00:00.000,Cattaraugus,2,12,16,251


In [3]:
#only kep 
df_tot_pos = results_df.drop(columns={'total_number_of_tests','cumulative_number_of_tests','new_positives'})
df_daily = results_df['new_positives']

In [4]:
#make a list of days (no duplicates)
days = results_df['test_date'].drop_duplicates()
days.head()
#days.reset_index()

0      2020-04-05T00:00:00.000
62     2020-04-04T00:00:00.000
124    2020-04-03T00:00:00.000
186    2020-04-02T00:00:00.000
248    2020-04-01T00:00:00.000
Name: test_date, dtype: object

In [5]:
date = {'date': [0]}

#Make empty dataframe

#Redo: iloc county column, remove duplicates
#--
#grab counties for any day/drop date column
df_temp = df_tot_pos.loc[results_df['test_date'] == days.iloc[0]]
df_temp = df_temp.drop(columns='test_date')
    
dft_temp = df_temp.T
#dft_temp
dft_temp = dft_temp.reset_index()

new_header = dft_temp.iloc[0] #grab the first row for the head
#print(new_header)
#--

#creat df, index as date
df = pd.DataFrame(columns = new_header)


df #62 counties in NYS

,county,Albany,Allegany,Bronx,Broome,Cattaraugus,Cayuga,Chautauqua,Chemung,Chenango,...,Sullivan,Tioga,Tompkins,Ulster,Warren,Washington,Wayne,Westchester,Wyoming,Yates


In [6]:
df.iloc[:,0]

Series([], Name: county, dtype: object)

In [7]:
#look for each day in date list
for i in range(len(days)):
    
    #data for i-th day
    today = days.iloc[i]
    print(today)
    
    #get total positive for i-th day from results df
    df_today = df_tot_pos.loc[results_df['test_date'] == days.iloc[i]]
    df_today.drop(columns='test_date',inplace=True)
    
    dft = df_today.T
    
    new_header = dft.iloc[0] #grab the first row for the header
    df_temp = dft[1:] #take the data less the header row
    df_temp.columns = new_header
    
    #df_temp.index.name = 'date'
    #df_temp.rename(index={: "x"},inplace=True)
    df_temp['dates'] = today 
    #print(df_temp)
    print()
    df = df.append(df_temp,ignore_index = False,sort=True)

#df.set_index('dates',inplace=True)

2020-04-05T00:00:00.000

2020-04-04T00:00:00.000

2020-04-03T00:00:00.000

2020-04-02T00:00:00.000

2020-04-01T00:00:00.000

2020-03-31T00:00:00.000

2020-03-30T00:00:00.000

2020-03-29T00:00:00.000

2020-03-28T00:00:00.000

2020-03-27T00:00:00.000

2020-03-26T00:00:00.000

2020-03-25T00:00:00.000

2020-03-24T00:00:00.000


/Users/ericoliverio/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/ericoliverio/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2020-03-23T00:00:00.000

2020-03-22T00:00:00.000

2020-03-21T00:00:00.000

2020-03-20T00:00:00.000



In [8]:
df #renma eto df_tot_pos or something 

,Albany,Allegany,Bronx,Broome,Cattaraugus,Cayuga,Chautauqua,Chemung,Chenango,Clinton,...,Tompkins,Ulster,Warren,Washington,Wayne,Westchester,Wyoming,Yates,county,dates
cumulative_number_of_positives,319,17,14327,76,12,11,15,47,46,36,...,94,372,26,19,34,14294,23,1,NaN,2020-04-05T00:00:00.000
cumulative_number_of_positives,305,16,13368,71,10,8,11,38,43,33,...,88,332,25,18,32,13723,22,1,NaN,2020-04-04T00:00:00.000
cumulative_number_of_positives,293,16,12315,65,9,7,10,36,39,31,...,85,290,20,16,30,13081,18,1,NaN,2020-04-03T00:00:00.000
cumulative_number_of_positives,267,14,11086,56,9,6,9,35,32,30,...,84,263,19,15,30,12351,15,1,NaN,2020-04-02T00:00:00.000
cumulative_number_of_positives,253,12,9927,46,8,4,8,22,28,25,...,74,240,18,12,27,11567,14,1,NaN,2020-04-01T00:00:00.000
cumulative_number_of_positives,240,9,8861,42,7,3,6,22,26,25,...,68,221,18,10,24,10683,10,0,NaN,2020-03-31T00:00:00.000
cumulative_number_of_positives,226,7,7766,38,6,3,6,20,19,21,...,66,211,18,10,19,9967,9,0,NaN,2020-03-30T00:00:00.000
cumulative_number_of_positives,217,7,6962,35,6,3,5,15,17,17,...,66,190,18,7,15,9326,8,0,NaN,2020-03-29T00:00:00.000
cumulative_number_of_positives,205,6,6304,29,4,2,5,15,15,13,...,52,146,18,7,12,8519,8,0,NaN,2020-03-28T00:00:00.000
cumulative_number_of_positives,195,2,5403,23,1,2,5,12,8,12,...,45,128,13,6,12,7875,7,0,NaN,2020-03-27T00:00:00.000


In [9]:
#what about NaN for earlier dates
#add NYS tot pos column
#is the Bronx/boroughs counties? How to consider NYC data

In [10]:
df = df.sort_values('dates',ascending=True)

In [11]:
from datetime import datetime

df.index = pd.to_datetime(df.index)

ValueError: ('Unknown string format:', 'cumulative_number_of_positives')

In [ ]:
df.Bronx

In [ ]:
df.index

In [ ]:
plt.plot_date(df.index,df.Bronx)
plt.xticks(rotation='vertical')

In [ ]:
#find today (make method)

#loop over all unique test_date
#dft = df_pos.loc[results_df['test_date'] == days.loc[0]]

#find today (make method)
#df_today = dft['test_date']
#dft.drop(columns='test_date',inplace=True)

#df = dft.T

#new_header = df.iloc[0] #grab the first row for the header
#df = df[1:] #take the data less the header row
#df.columns = new_header

#df.index.name = 'date'
#df.rename( index={'cumulative_number_of_positives': days.loc[0]}, inplace=True)
#append